In [1]:
%store -r x_tr #training data
%store -r y_tr #training labels
%store -r x_test #test data
%store -r y_test #test labels
%store -r x_val #validation data
%store -r y_val #validation labels

no stored variable #training data
no stored variable #training labels
no stored variable #test data
no stored variable #test labels
no stored variable #validation data
no stored variable #validation labels


In [2]:
import numpy as np
x_test = np.reshape(x_test, (x_test.shape[0],96,96,3)) # Reshape test data to 3D

print('Training data shape: ',x_tr.shape)
print('Training data labels: ',y_tr.shape)
print('Validation data shape:', x_val.shape)
print('Validation data labels:', y_val.shape)
print('Test data shape: ',x_test.shape)
print('Test data labels: ',y_test.shape)

Training data shape:  (30000, 96, 96, 3)
Training data labels:  (30000,)
Validation data shape: (7500, 96, 96, 3)
Validation data labels: (7500,)
Test data shape:  (12500, 96, 96, 3)
Test data labels:  (12500,)


In [3]:


import numpy as np
import pandas as pd
import cv2
import os
from glob import glob 
import matplotlib.pyplot as plt

import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, roc_auc_score

from keras_preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers import Conv2D, MaxPool2D

from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from IPython.display import clear_output
import tensorflow as tf



Using TensorFlow backend.


In [4]:
from keras.applications.resnet50 import ResNet50
import keras
from keras.layers import Dense, Activation, Dropout, BatchNormalization, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop, Adam
import numpy as np
np.random.seed(100)


dropout_fc = 0.5

conv_base = ResNet50(weights = 'imagenet', include_top = False, input_shape = (96,96,3))

model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(dropout_fc))
model.add(Dense(1, activation = "sigmoid"))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.5/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 3, 3, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               4718592   
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
activation_50 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
___________________________

In [5]:
model.compile(Adam(lr=0.0001), loss='binary_crossentropy', metrics = ['accuracy'])

In [6]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allocator_type = 'BFC'
config.gpu_options.per_process_gpu_memory_fraction = 0.40

In [6]:
model.fit(x_tr, y_tr, batch_size = 100, epochs = 10, verbose=1, validation_data= (x_val, y_val))

Instructions for updating:
Use tf.cast instead.
Train on 30000 samples, validate on 7500 samples
Epoch 1/10
30000/30000 [==============================] - 70s 2ms/step - loss: 0.3095 - acc: 0.8687 - val_loss: 0.2646 - val_acc: 0.9056
Epoch 2/10
30000/30000 [==============================] - 55s 2ms/step - loss: 0.1371 - acc: 0.9479 - val_loss: 0.2199 - val_acc: 0.9232
Epoch 3/10
30000/30000 [==============================] - 55s 2ms/step - loss: 0.0639 - acc: 0.9779 - val_loss: 0.3636 - val_acc: 0.8972
Epoch 4/10
30000/30000 [==============================] - 55s 2ms/step - loss: 0.0410 - acc: 0.9858 - val_loss: 0.3012 - val_acc: 0.9219
Epoch 5/10
30000/30000 [==============================] - 55s 2ms/step - loss: 0.0335 - acc: 0.9885 - val_loss: 0.2477 - val_acc: 0.9317
Epoch 6/10
30000/30000 [==============================] - 56s 2ms/step - loss: 0.0300 - acc: 0.9900 - val_loss: 0.2654 - val_acc: 0.9284
Epoch 7/10
30000/30000 [==============================] - 56s 2ms/step - loss: 0.

In [7]:
score = model.evaluate(x_test, y_test, verbose = 0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.2825796535246074
Test accuracy: 0.9293599999809266


In [4]:
from keras.preprocessing.image import ImageDataGenerator
roc_validation_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(x_val,
                                                                                  target_size=(96,96),
                                                                                  batch_size=100,
                                                                                  class_mode='binary',
                                                                                  shuffle=False)
predictions = model.predict_generator(roc_validation_generator, steps=len(roc_validation_generator), verbose=VERBOSITY)
false_positive_rate, true_positive_rate, threshold = roc_curve(roc_validation_generator.classes, predictions)
area_under_curve = auc(false_positive_rate, true_positive_rate)

plt.plot([0, 1], [0, 1], 'k--')
plt.plot(false_positive_rate, true_positive_rate, label='AUC = {:.3f}'.format(area_under_curve))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.savefig(ROC_PLOT_FILE, bbox_inches='tight')
plt.show()

NameError: name 'x_val' is not defined